# Course 2: Project - Data cleaning

This notebook is concerned with cleaning the [Open Food Facts](https://www.kaggle.com/openfoodfacts/world-food-facts) dataset (version 5), downloaded from Kaggle. The dataset originates from https://world.openfoodfacts.org/data. A description of the fields is available at https://static.openfoodfacts.org/data/data-fields.txt.

**Contents:**
* [Imports](#imports)
* [Preparatives](#preparatives)
* [A. Importing, cleaning and numerical summaries](#task-a)
  * [A.1. Column group: General information](#task-a1-general-information)
  * [A.1. Column group: Tags](#task-a1-tags)
  * [A.1. Column group: Ingredients](#task-a1-ingredients)

## Imports<a name="imports"></a>
---

In [5]:
# Standard library:
import collections
import enum
import functools
import pathlib
import re
import typing as t

# 3rd party:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Project:
import cleanquantity
import ean

%matplotlib inline

## Preparatives<a name="preparatives"></a>
---

This section regroups utility functions, etc. that we will use later in this notebook.

### Utilities

In [6]:
@functools.wraps(display)  # nicer for interactive use
def display_allcols(*args, **kwargs):
    """Behaves exactly like ``display`` but in a context where Pandas display all columns."""
    with pd.option_context('display.max_columns', None):
        display(*args, **kwargs)
        
        
@functools.wraps(display)  # nicer for interactive use
def display_allcols_notrunc(*args, **kwargs):
    """Behaves exactly like ``display`` but in a context where Pandas display all columns with no truncation."""
    with pd.option_context('display.max_columns', None, 'display.max_colwidth', -1):
        display(*args, **kwargs)


def profile(df: pd.DataFrame) -> pd.DataFrame:
    
    
    def get_type_name(obj: t.Any) -> str:
        return type(obj).__name__
        
    types_ = [df[col].map(get_type_name).value_counts().to_dict() for col in df.columns]
        
    data = {
        'Types': types_,
        'NA Count': df.apply(lambda series: series.isna().sum()),
        'NA %': df.apply(lambda series: series.isna().mean() * 100.0)
    }
    return pd.DataFrame(data).transpose()
    

def parse_t(series: pd.Series) -> pd.Series:
    return pd.to_datetime(series, utc=True, unit='s')


def parse_datetime(series: pd.Series) -> pd.Series:
    return pd.to_datetime(series, format='%Y-%m-%dT%H:%M:%S%z')


def move_after(words: t.List[str], word: str, word_to_move: str) -> t.List[str]:
    """Utility function to re-order columns."""
    try:
        word_idx = words.index(word)
        word_to_move_idx = words.index(word_to_move)
    except ValueError:
        pass
    else:
        if word_idx < word_to_move_idx:
            words.pop(word_to_move_idx)
            words.insert(word_idx + 1, word_to_move)
        else:
            words.insert(word_idx + 1, word_to_move)
            words.pop(word_to_move_idx)
    return words

### EAN-13/EAN-8/UPC-A

As per the description of the field `code`: For products with a barcode, this is the barcode of the product (EAN-13 code or some internal code assigned by the store). For products without a barcode, Open Food Facts assigns a number starting with the 200 reserved prefix. We implement utility functions to check whether a given code is a valid EAN-13/EAN-8/UPC-A code.

**Note:** In order not to clutter the notebook the code is in a separate module - *ean.py*.

## A. Importing, cleaning and numerical summaries<a name="task-a"></a>
---

Since we are not familiar with the dataset and warned that it is quite messy, we first let Pandas read the TSV file entirely into memory and guess the type of each column. As the end of this notebook, we will export a cleaned-up file that Pandas will be able to read more efficiently (e.g. by specifying the type of each column upfront).

In [8]:
data_filename = pathlib.Path.cwd().joinpath('en.openfoodfacts.org.products.tsv')

In [4]:
df = pd.read_csv(data_filename, sep='\t', low_memory=False)

We first get some general information about the `DataFrame`...

In [5]:
nrows, ncols = df.shape
print(f'the dataset contains {nrows} data points with {ncols} features')

the dataset contains 356027 data points with 163 features


**Note:** It turns out that reading the TSV file this way is problematic (at least on macOS) since 26 lines contain a carriage return. We noticed this by focusing on the first row where `code` was NaN and looking at the line corresponding to the previous row directly in the TSV file:
```bash
sed -n '193909 l' ./en.openfoodfacts.org.products.tsv
(...)fr-32-464-040-ec\t43.400279,0.199525\r\t\tvillecomtal-sur-arros-gers-france(...)
                                         ^^
```
26 data points is a negligible fraction of all data points and we could have dropped the rows but it turns out that there is an even simpler solution:

In [9]:
df = pd.read_csv(data_filename, sep='\t', lineterminator='\n', low_memory=False)

In [10]:
nrows, ncols = df.shape
print(f'the dataset contains {nrows} data points with {ncols} features')

the dataset contains 356001 data points with 163 features


Having taken care of this, we look at the first 5 rows:

In [11]:
display_allcols(df.head())

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,no_nutriments,additives_n,additives,additives_tags,additives_en,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutrition_grade_uk,nutrition_grade_fr,pnns_groups_1,pnns_groups_2,states,states_tags,states_en,main_category,main_category_en,image_url,image_small_url,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-butyric-acid_100g,-caproic-acid_100g,-caprylic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-lignoceric-acid_100g,-cerotic-acid_100g,-montanic-acid_100g,-melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-elaidic-acid_100g,-gondoic-acid_100g,-mead-acid_100g,-erucic-acid_100g,-nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
0,0000000003087,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,1474103866,2016-09-17T09:17:46Z,1474103893,2016-09-17T09:18:13Z,Farine de blé noir,NaN,1kg,NaN,NaN,Ferme t'y R'nao,ferme-t-y-r-nao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:FR,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"en:to-be-completed, en:nutrition-facts-to-be-c...","en:to-be-completed,en:nutrition-facts-to-be-co...","To be completed,Nutrition facts to be complete...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000004530,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,1489069957,2017-03-09T14:32:37Z,1489069957,2017-03-09T14:32:37Z,Banana Chips Sweetened (Whole),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,en:united-states,United States,"Bananas, vegetable oil (coconut oil, cor

For each column, we briefly look at the type guessed by Pandas and the number of non-null values:

In [9]:
df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356001 entries, 0 to 356000
Data columns (total 163 columns):
code                                          356001 non-null object
url                                           356001 non-null object
creator                                       355998 non-null object
created_t                                     356001 non-null int64
created_datetime                              356000 non-null object
last_modified_t                               356001 non-null int64
last_modified_datetime                        356001 non-null object
product_name                                  338489 non-null object
generic_name                                  57688 non-null object
quantity                                      119262 non-null object
packaging                                     89959 non-null object
packaging_tags                                89959 non-null object
brands                                        326977 non-null obj

**Comment:** We notice that 20+ columns contain only NA values.

**Decision:** Drop these columns.

In [10]:
df = df.dropna(how='all', axis=1)

In [11]:
nrows, ncols = df.shape
print(f'the dataset contains {nrows} data points with {ncols} features')

the dataset contains 356001 data points with 142 features


### A.1. Column group: General information<a name="task-a1-general-information"></a>

#### Column: `code`

In [12]:
profile(df[['code']])

,code
Types,{'str': 356001}
NA Count,0
NA %,0


**Comment:** The column `code` has type `object`, all values are of type `str` and there are no NA values.

**Decision:** Keep the column and all rows.

Out of curiosity, we check how many codes belong to the following categories: `e` - valid EAN-13/EAN-8/UPC-A code, `a` - code assigned by Open Food Facts (prefix 200) and `i` - internal code (store). (Mistyped EAN-13/EAN-8/UPC-A codes will be incorrectly classified as internal codes but we will not pursue this further.) We see that the majority of all codes (over 85 %) are valid EAN-13/EAN-8/UPC-A codes:

In [13]:
def categorize(code: str) -> str:
    return ('e' if ean.is_valid(code) else
            'a' if code.startswith('200') else
            'i')


df['code'].map(categorize).value_counts()

e    313911
i     40340
a      1750
Name: code, dtype: int64

#### Column: `url`

In [14]:
profile(df[['url']])

,url
Types,{'str': 356001}
NA Count,0
NA %,0


**Comment:** The column `url` has type `object`, all values are of type `str` and there are no NA values.

**Decision:** Keep the column and all rows.

#### Column: `creator`

In [15]:
profile(df[['creator']])

,creator
Types,"{'str': 355998, 'float': 3}"
NA Count,3
NA %,0.000842694


**Comment:** The column `creator` has type `object`, non-NA values are of type `str` and there are a neligible number of NA values.

**Decision:** Keep the column and all rows.

#### Columns: `created_(t,datetime)`, `last_modified_(t,datetime)`

In [16]:
profile(df[['created_t', 'created_datetime', 'last_modified_t', 'last_modified_datetime']])

,created_t,created_datetime,last_modified_t,last_modified_datetime
Types,{'int': 356001},"{'str': 356000, 'float': 1}",{'int': 356001},{'str': 356001}
NA Count,0,1,0,0
NA %,0,0.000280898,0,0


**Comment:** For the `created_t`/`created_datetime` pair: The column `created_t` has type `object`, all values are of type `str` and there are no NA values. Ditto for the column `created_datetime`, except that there is 1 NA value. We confirm that these columns agree where both are not NA:

In [17]:
s1 = parse_t(df['created_t'])
s2 = parse_datetime(df['created_datetime'])
both_notna = s1.notna() & s2.notna()
(s1[both_notna] == s2[both_notna]).all()

True

**Decision:** Use the column `created_t` to generate a column `created_on` (type `Timestamp`) and drop the columns `created_t` and `created_datetime`:

In [18]:
df['created_on'] = s1
df = df.drop(columns=['created_t', 'created_datetime'])

**Comment (cont.):** For the `last_modified_t`/`last_modified_datetime` pair: The column `last_modified_t` has type `object`, all values are of type `str` and there are no NA values. Ditto for the column `last_modified_datetime`. We confirm that these columns agree where both are not NA:

In [19]:
s1 = parse_t(df['last_modified_t'])
s2 = parse_datetime(df['last_modified_datetime'])
both_notna = s1.notna() & s2.notna()
(s1[both_notna] == s2[both_notna]).all()

True

**Decision (cont.):** Use the column `last_modified_t` to generate a column `last_modified_on` (type `Timestamp`) and drop the columns `last_modified_t` and `last_modified_datetime`:

In [20]:
df['last_modified_on'] = s1
df = df.drop(columns=['last_modified_t', 'last_modified_datetime'])

#### Column: `product_name`

In [21]:
profile(df[['product_name']])

,product_name
Types,"{'str': 338489, 'float': 17512}"
NA Count,17512
NA %,4.91909


**Comment:** The column `product_name` has type `object`, non-NA values are of type `str` and there are under 5 % of NA values.

**Decision:** Keep the column and all rows.

#### Column: `generic_name`

In [22]:
profile(df[['generic_name']])

,generic_name
Types,"{'float': 298313, 'str': 57688}"
NA Count,298313
NA %,83.7956


**Comment:** The column `generic_name` has type `object`, non-NA values are of type `str` and there are over 80 % of NA values. Inspecting a couple of records by hand, we notice that the language seems to vary quite a lot. Maybe tellingly, this column is not even documented.

**Decision:** Drop the column.

In [23]:
df = df.drop(columns='generic_name')

#### Column: `quantity`

In [24]:
profile(df[['quantity']])

,quantity
Types,"{'float': 236739, 'str': 119262}"
NA Count,236739
NA %,66.4995


**Comment:** The column `quantity` has type object, non-NA values are of type `str` and there are over 65 % of NA values. Inspecting a couple of entries by hand, we notice that, in most cases, the column indicates the quantity sold at once and the unit of measurement used. We also notice at least the following issues: a) Incorrect/incomplete entries (e.g. price in euros, product name, unitless quantity, etc.). b) Multiple languages are used (e.g. _320 г_ seems to mean 320 g in Russian). c) Mixed metric and imperial units.

**Decision:** We try to salvage as many of the non-NA values as possible as it might be interesting to know the quantity of product sold at once.

In order to keep complexity under control, we make the following assumptions:
* An entry must be a "valid number" followed by a "valid unit". White spaces are allowed and ignored. Additional information at the end is allowed and ignored. Letter case is ignored.
* A "valid number" is any string that matches `r'\d+(?:[.,]\d*)?'`.
* A "valid unit" is any string in `VALID_UNITS` (see code).
* Since imperial units differ between UK, US and USC, we decide to *ignore* those (see e.g. [How US labelling requirements undermine honest labelling in the UK](http://metricviews.org.uk/2013/03/how-us-labelling-requirements-undermine-honest-labelling-in-the-uk/).)

**Desired output:** 2 columns: `quantity_number` (type `float`) and `quantity_unit` (type `str`, either `g` or `l`). In the process, we convert all weights to gram and all volumes to liter.

**Note:** In order not to clutter the notebook, most of the code is in a separate module - _cleanquantity.py_.

In [25]:
ninitial = df['quantity'].notna().sum()

df_qty = cleanquantity.clean(df['quantity'])
df['quantity_number'] = df_qty['number']
df['quantity_unit'] = df_qty['unit']
df = df.drop(columns='quantity')

nstandardized = df_qty['number'].notna().sum()
pstandardized = nstandardized / ninitial * 100

print(f"entries initial: {ninitial}")
print(f"entries standardized: {nstandardized} ({pstandardized:.2f} % of not-NA)")

entries initial: 119262
entries standardized: 106470 (89.27 % of not-NA)


We check the result for outliers:

In [26]:
(df[['quantity_number', 'quantity_unit']]
 .groupby(by='quantity_unit')
 .describe())

quantity_number                                                \
                        count        mean          std  min     25%     50%   
quantity_unit                                                                 
g                     86658.0  403.239589  8037.160368  0.0  150.00  250.00   
l                     19812.0    0.809454     3.336295  0.0    0.35    0.75   

                                 
                 75%        max  
quantity_unit                    
g              420.0  1390000.0  
l                1.0      450.0

**Comment (cont.):** Weight: Inspecting a couple of entries above 10 kg by hand (22 entries), we notice the following: a) At least one record desribes food for *animals*, not for humans (code: 289259, categories: *aliment pour chevaux* (*food for horses*)). b) Some values do make sense (e.g. a 25 kg bag of flour for a baker) while others do not. Volume: Inspecting a couple of records above 12 liters by hand (12 entries), we notice that some values do makes sense (e.g. a 20 l barrel of wine) while others do not.

**Decision (cont.):** We have a negligible number of outliers. Weight: Replace a weigth of 0 by NA and drop records above 10 kg, except when `product_name` contains the word _farine_ (_flour_). Volume: Replace 0 by NA and drop records above 12 l except when `product_name` contains the word _tonneau_ (_barrel_).

In [27]:
df.loc[df['quantity_number'] == 0, ['quantity_number', 'quantity_unit']] = np.nan

weight_cond = ((df['quantity_unit'] == 'g')
          & (df['quantity_number'] > 10_000)
          & ~df['product_name'].str.contains('farine', case=False, na=False, regex=False))

volume_cond = ((df['quantity_unit'] == 'l')
          & (df['quantity_number'] > 12)
          & ~df['product_name'].str.contains('tonneau', case=False, na=False, regex=False))

df = df.drop(df[weight_cond | volume_cond].index, axis=0)

In [28]:
nrows, ncols = df.shape
print(f'the dataset contains {nrows} data points with {ncols} features')

the dataset contains 355971 data points with 140 features


### A.1. Column group: Tags<a name="task-a1-tags"></a>

#### Column: *first\_packaging\_code\_geo*

In [29]:
profile(df[['first_packaging_code_geo']])

,first_packaging_code_geo
Types,"{'float': 335103, 'str': 20868}"
NA Count,335103
NA %,94.1377


**Comment:** The column *first\_packaging\_code\_geo* has type object, non-NA values are of type `str` and there are  almost 95 % of NA values. We confirm that all non-NA entries are valid coordinates:

In [30]:
def are_valid_coordinates(text: str) -> bool:
    """\
    Checks whether a string contains valid geographic coordinates.
    """
    if pd.isna(text):
        return text
    
    tokens = text.split(',')
    if len(tokens) != 2:
        return False
    lat_token, lon_token = tokens
    try:
        lat = float(lat_token)
        lon = float(lon_token)
    except ValueError:
        return False
    if not (-90.0 <= lat <= 90.0):
        return False
    if not (-180.0 <= lon <= 180.0):
        return False
    return True


is_valid = df['first_packaging_code_geo'].map(are_valid_coordinates)
df['first_packaging_code_geo'].notna().sum() == is_valid.sum()

True

**Decision:** Keep the column and all rows.

### A.1. Column group: General information<a name="task-a1-general-information"></a>

In order to keep complexity under control, we make the following assumptions about a "properly formatted" list of ingredients:
* It is a delimited list of ingredients.
* The class of delimiters is `[.,;•]` (we do not support abbreviated words ending with a period).
* An ingredient is a sequence of words that may contains white spaces before, between and after the words.
* A word is a sequence of symbols which are not delimiters nor parentheses or a "number".
* Each ingredient is either a "simple" ingredient or a "composite" ingredient.
* A "composite" ingredient is followed by its own list of ingredients ("simple" or "composite") inside a pair of matching parentheses.
* Matching pairs of parentheses are `(` / `)` and `[` / `]`.

Recovery strategy:
* Use "panic mode" for the lexer.
```
# Words:
<space> ::= re('\s+') .
<delim> ::= '.' | ',' | ';' | '•' .
<l-par> ::= '(' | '[' .
<r-par> ::= ')' | ']' .

# Syntax:
<list> ::= <ingr> { <delim> <ingr> }* .
<ingr> ::= <field> [ <l-par> <list> <r-par> ] .
```

In [13]:
display_allcols(df.loc[[355945, 355948, 355951, 355952, 355953, 355954], :])

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,no_nutriments,additives_n,additives,additives_tags,additives_en,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutrition_grade_uk,nutrition_grade_fr,pnns_groups_1,pnns_groups_2,states,states_tags,states_en,main_category,main_category_en,image_url,image_small_url,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-butyric-acid_100g,-caproic-acid_100g,-caprylic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-lignoceric-acid_100g,-cerotic-acid_100g,-montanic-acid_100g,-melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-elaidic-acid_100g,-gondoic-acid_100g,-mead-acid_100g,-erucic-acid_100g,-nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
355945,9782091778181,http://world-en.openfoodfacts.org/product/9782...,kiliweb,1496600202,2017-06-04T18:16:42Z,1496600206,2017-06-04T18:16:46Z,La pratique du vocabulaire allemand,NaN,NaN,NaN,NaN,Nathan,nathan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,France,en:france,France,I • La grammajre françatse La communication or...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,[ i -> fr:i ] [ la-grammajre-francatse-la-c...,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,e,unknown,unknown,"en:to-be-completed, en:nutrition-facts-complet...","en:to-be-completed,en:nutrition-facts-complete...","To be completed,Nutrition facts completed,Ingr...",NaN,NaN,NaN,NaN,96.0,NaN,25.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.0,53.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,5.0,NaN,NaN,NaN,12.0,4.724409,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,28.0,NaN,NaN
355948,9782226399311,http://world-en.openfoodfacts.org/product/9782...,kiliweb,1505392509,2017-09-14T12:35:09Z,1505392514,2017-09-14T12:35:14Z,Treize Raisons,NaN,NaN,NaN,NaN,netflix,netflix,en:milks,"en:dairies,en:milks","Dairies,Milks",NaN,NaN,NaN,N

In [423]:
# df2 = df[df['countries_tags'].notna()][['countries', 'countries_tags']]
# df3 = df2['countries_tags'].str.extract(re.compile(r'^(?P<lang2>..)\:(?P<name>.*)$'))
# display(
#     df3['lang2'].value_counts(),
#     df3['name'].value_counts()
# )

en = ('united-states', 'united-kingdom', 'australia')  # leave out canada
fr = ('france', 'switzerland', 'belgium')  # leave out canada
pattern_en = re.compile('|'.join(re.escape(name) for name in en))
pattern_fr = re.compile('|'.join(re.escape(name) for name in fr))
has_en = df['countries_tags'].str.contains(pattern_en, na=False, regex=True)
has_fr = df['countries_tags'].str.contains(pattern_fr, na=False, regex=True)

display(
    df[has_en]['ingredients_text'].notna().sum(),
    df[has_fr]['ingredients_text'].notna().sum()
)

#df[has_en][['countries_tags', 'ingredients_text']]
#df[has_fr][['countries_tags', 'ingredients_text']]

175494

95590

In [376]:
def to_pairs(tokens: t.Iterable[Token]) -> t.List[t.Tuple[str, bool]]:
    result = []
    state = ParseState(next(tokens), tokens, [])
    cur = state.peek
    while cur.type != TokenType.END:
        state = read(state)
        nxt = state.peek
        
        if cur.type == TokenType.FIELD:
            text = cur.text
            result.append( (text, state.peek.type == TokenType.LPAR) )
        
        cur = nxt
    return result

In [380]:
import unicodedata


def remove_accents(text: str) -> str:
    """\
    DOCME
    """
    normalized = unicodedata.normalize('NFKD', text)
    return "".join(c for c in normalized if not unicodedata.combining(c))


def replace_punctuation(text: str, by: str) -> str:
    """\
    DOCME
    """
    return "".join(by if unicodedata.category(c).startswith('P') else c for c in text)


def normalize(text: str) -> str:
    return remove_accents(replace_punctuation(text.strip().lower(), by=''))

In [422]:
# import nltk
# nltk.download('punkt')

from nltk.tokenize import word_tokenize, sent_tokenize
#sent_tokenize(text)
#word_tokenize(text)

In [383]:
def try_to_parse(series: pd.Series) -> t.Tuple[str, int]:
    field = series['ingredients_text']
    
    if pd.isna(field):
        return field, None, None
    
    tokens = tokenize(field)
    state = ParseState(next(tokens), tokens, [])
    state, result = parse_list(state)
    processed = flatten(merge(result))
    normalized = [(normalize(t), b) for t, b in processed]
    return field, str(normalized), len(state.errors)


result = df.head(100000)[['ingredients_text']].apply(try_to_parse, axis='columns', result_type='expand')
display(
    result[2].isna().sum(),
    (result[2] > 0).sum()
)
# display_allcols_notrunc(result)
display_allcols_notrunc(result[result[2] > 0])

2882

813

,0,1,2
471,"Wheatflour contains Gluten (With Wheatflour, Calcium Carbonate, Iron, Niacin, nut) Thiamin) Dark Chocolate Chips (20%) (Sugar ? Cocoa Butter ? Cocoa Mass ? Emulsifier: Soya Lecithin ? Vanilla llavouri ng) ? Palm Oil ? Sugar ? Roasted Hav!nuts (3%) ? Dried }kimmed Milk ? Golden Syrup (Invert Sugar Syrup) Salt ? Raising Agent: Sodium Bicarbonate, E450, E503 ? Emulsifier: Soya Lecithin ? Vanilla Flavouring. Dark Chocolate contains Cocoa Solids 39% minimum.","[('wheatflour contains gluten', True), ('with wheatflour', False), ('calcium carbonate', False), ('iron', False), ('niacin', False), ('nut', False), ('thiamin', False), ('dark chocolate chips', True), ('20', False), ('sugar', False), ('cocoa butter', False), ('cocoa mass', False), ('emulsifier soya lecithin', False), ('vanilla llavouri ng', False), ('palm oil', False), ('sugar', False), ('roasted havnuts', True), ('3', False), ('dried kimmed milk', False), ('golden syrup', True), ('invert sugar syrup', False), ('salt', False), ('raising agent sodium bicarbonate', False), ('e450', False), ('e503', False), ('emulsifier soya lecithin', False), ('vanilla flavouring', False), ('dark chocolate contains cocoa solids 39 minimum', False)]",1.0
559,",tato crisps With mint and salt marsh lamb Seasoning GREDIENTS Potatoes • Sunflower Oil • Rice Flour • Igar • Salt • Dried Yeast Extract • Dried Onions . Dried der Vinegar • Dried Garlic • Dried Mint Flavouring • itish Salt Marsh Lamb Seasoning (Salt • Lamb Fat • )semary Extract) • Ground Black Pepper • Acidity Citric Acid • Colour: Paprika Extract. NFORMATION This product is sold by weight. }ettling of the contents may occur.","[('tato crisps with mint and salt marsh lamb seasoning gredients potatoes', False), ('sunflower oil', False), ('rice flour', False), ('igar', False), ('salt', False), ('dried yeast extract', False), ('dried onions', False), ('dried der vinegar', False), ('dried garlic', False), ('dried mint flavouring', False), ('itish salt marsh lamb seasoning', True), ('salt', False), ('lamb fat', False), ('semary extract', False), ('ground black pepper', False), ('acidity citric acid', False), ('colour paprika extract', False), ('nformation this product is sold by weight', False), ('ettling of the contents may occur', False)]",1.0
573,Apple Juice (50%) Coconut Water 15%) ? Cucumber Purée (15%) Apple Purée 14%) ? Spinach Purée (3%) ? Lemon Juice ? Spirulina Extract ? Antioxidant: Ascorbic Acid.,"[('apple juice', True), ('50', False), ('coconut water 15', False), ('cucumber puree', True), ('15', False), ('apple puree 14', False), ('spinach puree', True), ('3', False), ('lemon juice', False), ('spirulina extract', False), ('antioxidant ascorbic acid', False)]",2.0
698,"Jus de pomme pressée pasteurisé ? Purée de banane ? Purée de pomme ? Purée dei cassis (10%) ? Puree de framboise 1096) ? Purée de baie dlaçaï ? correcteur d'acidité : Acide citrique ? . Antioxydant : Acide ascorbique. :iNFORMATlON Ce produit peut se idissocier naturellement. INGREDIÈNTEN ) Gepasteuriseerd geperst àppelsap ? bananenpuree ? làppelpuree ? zwafte bessenpuree S) ? frambozenpuree (10%) ? bessenpuree ? zuufteregelaar: :titrcenzuur ? antioxidant: Èscorbinezuur. INFORMATE Dit product kan van nature bezinken. movennes/Gemiddelde waarden : : ter -00ml: Energie/Energie 3ò6KJ/87kcal ? Matières grasses/ l'etten 1.9g, dont acides gras saturés/ Waawan verzadigde vetzuren 1.0g ? G!uc des/Koolhydraten 14.8g, dont Sucres/waarvan suikers 9.4g ? Fibres à)mentairesNezels 0.6g ? Protéines/ 2.3g ? Sel/Zout 0.05g. 5)65. looml 25(31) per 500ml 125(1 : = Valeurs Nutritionnelles a Référence/ VRI Dagelijkse Referentie-lnnanne BEST BEFORE","[('jus de pomme pressee pasteurise', False), ('puree de banane', False), ('puree de pomme', False), ('puree dei cassis', True), ('10', False), ('puree de framboise 1096', False), ('puree de baie dlacai', False), ('correcteur dacidite acide citrique', False), ('antioxydant acide ascorbique', False), ('informatlon ce produit peut se i

In [ ]:
# Unbalanced parenthesis:
s = '''Peanut butter coating (evaporated cane juice, fractionated palm kernal oil, partially defatted peanut flour, whey powder [milk], soy lecithin [an emulsifier]), milk chocolate coating (dehydrated cane juice, cocoa butter, unsweetened chocolate, whole milk powder, soy lecithin [an emulsifier], natural vanilla), malt balls (glucose syrup [corn], whey powder, malted milk powder [malted barley, wheat flour, milk, bicarbonate of soda, mono and diglycerides), pure food glaze.'''
#test_parse(s)
#list(get_tokens(s.lower()))

In [ ]:
display_allcols_notrunc(df[df.iloc[:, 0:30].notna().all(axis='columns')])

Specification:
* Lower-case
* Split on '.' -> groups
* Split group on ','
* If ends with '... (...) for the moment ignore

In [ ]:
s_ingredients = df['ingredients_text']


def clean_ingredients_text(text: str) -> t.List[str]:
    if pd.isna(text):
        return text
    
    # Strip leading/trailing white-spaces:
    text = text.strip()
    # Convert to lower case:
    text = text.lower()
    # Remove trailing period, if any:
    if text.endswith('.'):
        text = text[:-1]
    words = text.split(',')
    words = [word.strip() for word in words]
        
    return '|'.join(words)
    
    
s2 = s_ingredients.map(clean_ingredients_text)
df['tmp'] = s2
display_allcols_notrunc(s2)

# TAGS / WHAT TO DO?

In [ ]:
BASE_COLUMNS = (
    'packaging',
    'brands',
    'categories',
    'origins',
    'manufacturing_places',
    'labels',
    'emb_codes',
    'cities',
    'countries'
)

# To investigage:
# - purchase_places
# - stores

def get_related_tags_columns(df: pd.DataFrame, base_column: str) -> t.List[str]:
    result = []
    for column in df.columns:
        if column == base_column:
            result.append(column)
        elif re.match(fr'{re.escape(base_column)}_tags', column):
            result.append(column)
        elif re.match(fr'{re.escape(base_column)}_[a-z]{{2}}', column):
            result.append(column)
    return result

In [ ]:
#display_allcols(df[df['first_packaging_code_geo'].notna()])
#display_allcols(df[df['purchase_places'].notna()])


In [ ]:
get_related_tags_columns(df, 'categories')

Plan for tags:
* Lower-case the text, split the text on white-spaces (remove leading/trailing/repeated white-spaces), remove accents for all words, join the words with dashes.
* In a 2-language column, filter out tags with a "foreign" prefix. E.g. filter out fr:Foies tags in categories_en.

Example: (_code_: `452`, *product\_name* : `Foie gras canard Périgord`)
* _categories_: `Foie gras de canard`
* *categories\_tags*: `en:fish-and-meat-and-eggs,fr:foies-gras,fr:foies-gras-de-canard`
* *categories\_en*: `Fish and meat and eggs,fr:Foies gras,fr:Foies gras de canard`

Expected output:
* _categories_: `foie-gras-de-canard`
* *categories\_tags*: `en:fish-and-meat-and-eggs,fr:foies-gras,fr:foies-gras-de-canard` (no change)
* *categories\_en*: `fish-and-meat-and-eggs`

Hypothesis for tags:
* `<name>` is a comma-separated list of short descriptions in one of the languages of the country where the product was bought. These keywords may include white spaces and seem to start with an upper case letter.
* `<name>_tags_en` is the translation of `<name>` to English.
* `<name>_tags` is a comma-separated list of tags, where tags are derived from the keywords by lower-casing all words and joining them with dashes. Each tag is optionally prefixed with a language 2-letter code followed by a colon.

**Comment:** The number of tags does not seem to agree. E.g. for categories sometimes 

In [ ]:
# Split 'categories' into a list of short descriptions:
s1 = df['categories'].map(lambda text: text.split(','), na_action='ignore')
# Count the number of short descriptions:
ns1 = s1.map(lambda keywords: len(keywords), na_action='ignore')

# Split 'categories_en' into a list of short descriptions:
s2 = df['categories_en'].map(lambda text: text.split(','), na_action='ignore')
# Count the number of short descriptions:
ns2 = s2.map(lambda keywords: len(keywords), na_action='ignore')

# Split 'categories_tags' into a list of tags:
s3 = df['categories_tags'].map(lambda text: text.split(','), na_action='ignore')
# Count the number of tags:
ns3 = s3.map(lambda tags: len(tags), na_action='ignore')

df_tmp = pd.DataFrame(data={
    'categories': s1, 'categories_len': ns1,
    'categories_en': s2, 'categories_en_len': ns2,
    'categories_tags': s3, 'categories_tags_len': ns3
})

display(
    (df_tmp['categories_len'] < df_tmp['categories_en_len']).sum(),
    (df_tmp['categories_len'] > df_tmp['categories_en_len']).sum(),
    (df_tmp['categories_len'] == df_tmp['categories_en_len']).sum(),
    '---',
    (df_tmp['categories_len'] < df_tmp['categories_tags_len']).sum(),
    (df_tmp['categories_len'] > df_tmp['categories_tags_len']).sum(),
    (df_tmp['categories_len'] == df_tmp['categories_tags_len']).sum(),
    '---',
    (df_tmp['categories_en_len'] < df_tmp['categories_tags_len']).sum(),
    (df_tmp['categories_en_len'] > df_tmp['categories_tags_len']).sum(),
    (df_tmp['categories_en_len'] == df_tmp['categories_tags_len']).sum()
)

# display(
#     (df_tmp['categories_len'] != df_tmp['categories_en_len']).sum(),
#     (df_tmp['categories_len'] != df_tmp['categories_tags_len']).sum(),
#     (df_tmp['categories_en_len'] != df_tmp['categories_tags_len']).sum()
# )

# display_allcols_notrunc(df_tmp[(
#     df_tmp['categories'].notna() &
#     ~((df_tmp['categories_len'] == df_tmp['categories_en_len']) & (df_tmp['categories_len'] == df_tmp['categories_tags_len'])))
# ])

In [ ]:
columns = [
    'packaging', 'packaging_tags',
    'brands', 'brands_tags',
    'categories', 'categories_tags', 'categories_en',
    'origins', 'origins_tags',
    'manufacturing_places', 'manufacturing_places_tags',
    'labels', 'labels_tags', 'labels_en',
    'emb_codes', 'emb_codes_tags'
]
profile(df[columns])

To investigage:
* _first_packaging_code_geo_
* *purchase\_places*
* _stores_

In [ ]:
display_allcols_notrunc(df.loc[[392,353516, 353517], :])

In [ ]:
#display_allcols(df[df['first_packaging_code_geo'].notna()])
#display_allcols(df[df['purchase_places'].notna()])
#display_allcols(df[df['stores'].notna()])
#display_allcols(df[df['countries'].notna()])
display_allcols_notrunc(df[df['emb_codes'].notna()])